# Firms population


In [1]:
#Load modules

import pandas as pd
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
from numpy import log10
import numpy as np

import warnings
warnings.filterwarnings('ignore')

## Load data
### Here we load the data aggregated at (firm, year, flow) level. The only variable we know for each (firm, year, flow) is the 'VART_sum', that is, the sum of transaction values at border.

In [3]:

save_path = './'

firm_mx_y = pd.read_csv(save_path + 'firm_sizes_99.csv')

In [4]:
#Sample of the loaded dataset

ix = ['ID', 'IMPORT', 'YEAR']
display(firm_mx_y.sort_values(by = ix).head(20).set_index(ix))
print '...'

VART
ID  IMPORT YEAR          
0   0      1997  39913032
           1998  25985808
    1      1997  46236989
           1998  29903677
18  1      1998      2047
157 1      1998     77846
215 0      1997    656617
           1998    807502
    1      1997      1292
223 1      1997    339981
           1998    257173
314 1      1998    121295
330 0      1997     23943
           1998     14937
355 0      1998     20277
    1      1997     37676
           1998     17312
371 1      1997    233617
           1998      3672
397 0      1997    190830

...


### Join Firms name

In [17]:
pd.options.display.max_columns = 999
pd.options.display.max_rows = 200

#Firms data (name, activity, employees)

firms = pd.read_csv('./../data/SIREN.csv', delimiter = ';', encoding = 'latin-1')
firms = firms[['SIREN','L1_NORMALISEE','LIBTEFET','APET700','LIBAPET']].set_index('SIREN')
firms.index.name = 'ID'

In [18]:
# df = pd.read_csv(path1+'/firm_lev_sample.csv')
df = pd.read_csv(save_path + 'firm_sizes_99.csv')

# 4 year period
df['PERIOD'] = (df.YEAR - 1997) % 4

n = df[['YEAR', 'PERIOD']].drop_duplicates().groupby('PERIOD').count().reset_index()
n.columns = ['PERIOD', 'n']

# drop year 1997, so that we got 4 periods of four years

df = df.loc[df.YEAR != 1997]

# Information on Combined nomenclature product codes (CN) and CPA activity codes
# CN_full = pd.read_csv('./../data/CN_full.csv', encoding = 'utf-8')
# df = df.merge(CN_full[['CN ID 8', 'CN ID 4', 'CN label 4']])

df.sample(5)

,ID,IMPORT,YEAR,VART,PERIOD
468782,319264552,1,1998,78684,1
380511,302517305,0,1998,1837,1
334917,318799335,0,1998,494441,1
369467,401067137,1,1998,18996,1
404458,398426775,0,1998,43786,1


In [19]:

dataset = df.loc[df.PERIOD == 1]

df_ = dataset.groupby(['ID', 'IMPORT']).agg({'VART': ['count','sum']}).sort_values(by = ('VART', 'sum'), ascending = False)
largest = df_.astype(float).merge(firms, left_index = True, right_index = True)
largest.head(20)

(VART, count)   (VART, sum)  \
ID        IMPORT                                
780129987 0                 1.0  1.341055e+10   
          1                 1.0  5.497488e+09   
602025538 1                 1.0  2.953639e+09   
542107651 1                 1.0  2.894874e+09   
542034921 1                 1.0  2.811562e+09   
552081317 0                 1.0  2.807210e+09   
319632790 0                 1.0  2.042990e+09   
855200507 0                 1.0  1.834658e+09   
954503439 0                 1.0  1.793718e+09   
780130175 1                 1.0  1.600560e+09   
622044287 1                 1.0  1.566649e+09   
552118465 1                 1.0  1.500865e+09   
331355263 1                 1.0  1.495457e+09   
954506077 0                 1.0  1.336187e+09   
552118465 0                 1.0  1.331453e+09   
341459386 0                 1.0  1.244849e+09   
542010053 1                 1.0  1.185964e+09   
342439320 1                 1.0  1.117174e+09   
305493173 1                 1.0  1.108748e+09   
542034327 1                 1.0  1.054063e+09   

                                           L1_NORMALISEE  \
ID        IMPORT                                           
780129987 0                                  RENAULT SAS   
          1                                  RENAULT SAS   
602025538 1             VOLKSWAG GROUP AUT RETAIL FRANCE   
542107651 1                                        ENGIE   
542034921 1                     TOTAL MARKETING SERVICES   
552081317 0                        ELECTRICITE DE FRANCE   
319632790 0                                ARKEMA FRANCE   
855200507 0               MANUF FRANC PNEUMATIQ MICHELIN   
954503439 0            SCHNEIDER ELECTRIC INDUSTRIES SAS   
780130175 1                   SOCIETE DES PETROLES SHELL   
622044287 1                         MERCEDES BENZ FRANCE   
552118465 1                               CIE IBM FRANCE   
331355263 1       SOC NAL EXPLOIT INDUST TABAC ALLUMETTE   
954506077 0                               RENAULT TRUCKS   
552118465 0                               CIE IBM FRANCE   
341459386 0                        STMICROELECTRONICS SA   
542010053 1               ESSO SOCIETE ANONYME FRANCAISE   
342439320 1                                  OPEL FRANCE   
305493173 1                                   FCA FRANCE   
542034327 1                                    BP FRANCE   

                                 LIBTEFET APET700  \
ID        IMPORT                                    
780129987 0            500 à 999 salariés   2910Z   
          1            500 à 999 salariés   2910Z   
602025538 1            500 à 999 salariés   4511Z   
542107651 1        1 000 à 1 999 salariés   3523Z   
542034921 1        1 000 à 1 999 salariés   7010Z   
552081317 0                     0 salarié   3511Z   
319632790 0            500 à 999 salariés   7010Z   
855200507 0       10 000 salariés et plus   2211Z   
954503439 0            500 à 999 salariés   2712Z   
780130175 1            500 à 999 salariés   4671Z   
622044287 1            500 à 999 salariés   4511Z   
552118465 1        2 000 à 4 999 salariés   6202A   
331355263 1            250 à 499 salariés   1200Z   
954506077 0        2 000 à 4 999 salariés   2910Z   
552118465 0        2 000 à 4 999 salariés   6202A   
341459386 0            250 à 499 salariés   7010Z   
542010053 1            250 à 499 salariés   4730Z   
342439320 1            100 à 199 salariés   4511Z   
305493173 1            250 à 499 salariés   4511Z   
542034327 1            250 à 499 salariés   7010Z   

                                                            LIBAPET  
ID        IMPORT                                                     
780129987 0                   Construction de véhicules automobiles  
          1                   Construction de véhicules automobiles  
602025538 1       Commerce de voitures et de véhicules automobil...  
542107651 1           Commerce de combustibles gazeux par conduites  
542034921 1                    

In [20]:
row_list = []
for ix in largest.head(100).index:
    dfv = dataset.loc[(dataset.ID == ix[0]) &(dataset.IMPORT == ix[1]) ].groupby('CN label 4').sum()[['VART']].sort_values(by = 'VART', ascending = False)
    dfx = (100*dfv/dfv.sum()).round()
    dfxv_p = pd.concat([dfv, dfx], axis = 1)
    dfxv_p.columns = ['VART', 'VART_pct']
    
    dfv = dataset.loc[(dataset.ID == ix[0]) &(dataset.IMPORT == ix[1]) ].groupby('PYOD').sum()[['VART']].sort_values(by = 'VART', ascending = False)
    dfx = (100*dfv/dfv.sum()).round()
    dfxv_c = pd.concat([dfv, dfx], axis = 1)
    dfxv_c.columns = ['VART', 'VART_pct']

    try: 
        row_list += [pd.DataFrame(pd.concat([pd.DataFrame({'ID':[ ix[0]], 'IMPORT': [ix[1]]}), 
                                            dfxv_p[['VART_pct']].reset_index().iloc[[0]],
                                            dfxv_c[['VART_pct']].reset_index().iloc[[0]]], axis = 1)).set_index('ID')]
    except:
        pass
    
firm_info = pd.concat(row_list)
firm_info.columns = ['IMPORT','main CN4', 'main CN4 pct','main PYOD', 'main PYOD pct']
firm_info = firm_info.reset_index().set_index(['ID', 'IMPORT'])

KeyError: 'CN label 4'

In [ ]:
display(HTML("<style>.container { width:100% !important; }</style>"))

# pd.set_option('display.float_format', '{:.2g}'.format)

res = df_.merge(firm_info, left_index=  True, right_index = True).sort_values(by = ('VART', 'sum'), ascending = False)
res = res.reset_index().set_index('ID').merge(firms, left_index = True, right_index = True).sort_values(by = ('VART', 'sum'), ascending = False)
res[('VART', 'sum')] = res[('VART', 'sum')].astype(float)
res


In [ ]:
import webbrowser

for siren in df_.index[30:60]:
    # google siren code
    webbrowser.open('https://www.google.com/search?q='+str(siren)+'+siren+score3')



## Other stuff

In [ ]:
grouped = df2012_sam_1.loc[df2012_sam_1.ID == 542034921].groupby('CN ID 8')

df_1 = grouped.sum()[['VFTE', 'VART']].astype(float)#.style.bar(color='#d65f5f')

df_2 = grouped.nunique()[['MONTH', 'DEPT', 'PYOD', 'CPA6','PAYP']]#.style.bar(color='#d65f5f')

pd.concat([df_1, df_2], axis = 1).sort_values(by = 'VART', ascending = False).style.bar(color='#d65f5f')

In [ ]:
df = df2012_sam_1

plt.hist(np.log10(df[13].values), 50)
plt.show()

In [ ]:
df_95['log_bin'] = np.log10(df_95[13]).round(2)
df_95.groupby('log_bin').sum()[13].plot(marker = 'o')

In [ ]:
df_95.loc[df_95[2].isin([1, 3]), 13].sum()
df_95.loc[df_95[2].isin([2, 4]), 13].sum()

In [ ]:

print true_divide(df_9999.loc[:, 12:].sum(), df_9.loc[:, 12:].sum())
print true_divide(df_999.loc[:, 12:].sum(), df_9.loc[:, 12:].sum())
print true_divide(df_99.loc[:, 12:].sum(), df_9.loc[:, 12:].sum())

In [ ]:
# df = pd.read_csv(path1+'/DP1610_MAASTRICHT1_2012.txt', delimiter = ';', header = None, nrows = 3000000)
# d = dict(df.dtypes)

# df = pd.read_csv(path1+'/DP1610_MAASTRICHT1_2012.txt', delimiter = ';', header = None,
#                 usecols=range(14), skiprows=lambda i: i % n != 0, low_memory=True, dtype = d)
df.columns = [u'YEAR', u'MONTH', u'FLUX', u'ID', u'DEPT', u'CN ID 8', u'CPA6',
       u'PYOD', u'PAYP', u'VAT', u'PRIFAC', u'DEVFAC', u'VFTE', u'VART']

print len(df)
df1_2012 = reduce_mem(df)

print(mem_usage(df))
print(mem_usage(df1_2012))

df1_2012

In [ ]:
# drafts from now on

In [ ]:
df1_2012 = pd.read_csv('./data/type1/DP1610_MAASTRICHT1_2012.txt', delimiter = ';', header = None)
pd.read_csv('./data/type1/DP1610_MAASTRICHT1_2013.txt', delimiter = ';', header = None)

In [ ]:
df2_2012 = reduce_mem(pd.read_csv('./data/type2/DP1611_MAASTRICHT2_2012.txt', delimiter = ';', header = None))
print(mem_usage(df2_2012_o))



# df2_2013 = pd.read_csv('./data/type2/DP1611_MAASTRICHT2_2013.txt', delimiter = ';', header = None)

In [ ]:
l = [df2_2012, df2_2013]

In [ ]:
for df in l:
    print len(df)

In [ ]:


gl = df2_2013

gl_int = gl.select_dtypes(include=['int'])
converted_int = gl_int.apply(pd.to_numeric,downcast='unsigned')

print(mem_usage(gl))
print(mem_usage(gl_int))
print(mem_usage(converted_int))

compare_ints = pd.concat([gl_int.dtypes,converted_int.dtypes],axis=1)
compare_ints.columns = ['before','after']
compare_ints.apply(pd.Series.value_counts)

In [ ]:
for df in l:
    display(df.sample(10))
    print df.max()